# Phase 1: Local Training Orchestration (Google Colab Compatible)

This notebook orchestrates all training activities for **local execution** with Google Colab GPU compute support.

## Overview

- **Step 1**: Load Centralized Configs
- **Step 2**: Verify Local Dataset (from data config)
- **Step 3**: Setup Local Environment
- **Step 4**: The Dry Run
- **Step 5**: The Sweep (HPO) - Local with Optuna
- **Step 6**: Best Configuration Selection (Automated)
- **Step 7**: Final Training (Post-HPO, Single Run)
- **Step 8**: Model Conversion & Optimization

## Important

- This notebook **executes training locally** (not on Azure ML)
- All computation happens on the local machine or Google Colab GPU
- The notebook must be **re-runnable end-to-end**
- Uses the dataset path specified in the data config (from `config/data/*.yaml`), typically pointing to a local folder included in the repository (no external downloads needed)


## Create a local Conda environment


### 1. Open a terminal in the project root

In PowerShell:

```powershell
cd "C:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml"
```

### 2. Create the Conda environment from the project’s `conda.yaml`

```powershell
conda env create -f config\environment\conda.yaml
```

- This will create an environment named `resume-ner-training` (from the `name:` field in the YAML).
- It installs Python 3.10, PyTorch, transformers, Azure ML SDK, etc.

If Conda says the env already exists, use:

```powershell
conda env update -f config\environment\conda.yaml
```

### 3. Activate the environment

```powershell
conda activate resume-ner-training
```

## Step P1-3.1: Load Centralized Configs

Load and validate all configuration files. Configs are immutable and will be logged with each job for reproducibility.


In [ ]:
import os
import sys
from pathlib import Path

IN_COLAB = "COLAB_GPU" in os.environ or "COLAB_TPU" in os.environ

# Assume this notebook lives in `notebooks/` under the project root
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
SRC_DIR = ROOT_DIR / "src"
CONFIG_DIR = ROOT_DIR / "config"

sys.path.append(str(ROOT_DIR))
sys.path.append(str(SRC_DIR))

print("Notebook directory:", NOTEBOOK_DIR)
print("Project root:", ROOT_DIR)
print("Source directory:", SRC_DIR)
print("Config directory:", CONFIG_DIR)
print("In Colab:", IN_COLAB)

In [ ]:
from pathlib import Path
from typing import Any, Dict

from orchestration import EXPERIMENT_NAME
from orchestration.config_loader import (
    ExperimentConfig,
    compute_config_hashes,
    create_config_metadata,
    load_all_configs,
    load_experiment_config,
    snapshot_configs,
    validate_config_immutability,
)

# P1-3.1: Load Centralized Configs (local-only)
# Mirrors the Azure orchestration notebook, but does not create an Azure ML client.

if not CONFIG_DIR.exists():
    raise FileNotFoundError(f"Config directory not found: {CONFIG_DIR}")

experiment_config: ExperimentConfig = load_experiment_config(CONFIG_DIR, EXPERIMENT_NAME)
configs: Dict[str, Any] = load_all_configs(experiment_config)
config_hashes = compute_config_hashes(configs)
config_metadata = create_config_metadata(configs, config_hashes)

# Immutable snapshots for runtime mutation checks
original_configs = snapshot_configs(configs)
validate_config_immutability(configs, original_configs)

print(f"Loaded experiment: {experiment_config.name}")
print("Loaded config domains:", sorted(configs.keys()))
print("Config hashes:", config_hashes)
print("Config metadata:", config_metadata)

# Get dataset path from data config (centralized configuration)
# The local_path in the data config is relative to the config directory
data_config = configs["data"]
local_path_str = data_config.get("local_path", "../dataset")
DATASET_LOCAL_PATH = (CONFIG_DIR / local_path_str).resolve()

# Check if seed-based dataset structure (for dataset_tiny with seed subdirectories)
seed = data_config.get("seed")
if seed is not None and "dataset_tiny" in str(DATASET_LOCAL_PATH):
    DATASET_LOCAL_PATH = DATASET_LOCAL_PATH / f"seed{seed}"


print(f"Dataset path (from data config): {DATASET_LOCAL_PATH}")
if seed is not None:
    print(f"Using seed: {seed}")


In [ ]:
# Use HPO config already loaded in configs (from Step P1-3.1)
hpo_config = configs["hpo"]
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]


## Step P1-3.2: Verify Local Dataset

Verify that the dataset directory (specified by `local_path` in the data config) exists and contains the required files. The dataset path is loaded from the centralized data configuration in Step P1-3.1.


In [ ]:
# P1-3.2: Verify Local Dataset
# The dataset path comes from the data config's local_path field (loaded in Step P1-3.1).
# This ensures the dataset location is controlled by centralized configuration.
# Note: train.json is required, but validation.json is optional (matches training script behavior).

REQUIRED_FILE = "train.json"
OPTIONAL_FILE = "validation.json"

if not DATASET_LOCAL_PATH.exists():
    raise FileNotFoundError(
        f"Dataset directory not found: {DATASET_LOCAL_PATH}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create the dataset, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check required file
train_file = DATASET_LOCAL_PATH / REQUIRED_FILE
if not train_file.exists():
    raise FileNotFoundError(
        f"Required dataset file not found: {train_file}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create it, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check optional file
val_file = DATASET_LOCAL_PATH / OPTIONAL_FILE
has_validation = val_file.exists()

print(f"✓ Dataset directory found: {DATASET_LOCAL_PATH}")
print(f"  (from data config: {data_config.get('name', 'unknown')} v{data_config.get('version', 'unknown')})")

train_size = train_file.stat().st_size
print(f"  ✓ {REQUIRED_FILE} ({train_size:,} bytes)")

if has_validation:
    val_size = val_file.stat().st_size
    print(f"  ✓ {OPTIONAL_FILE} ({val_size:,} bytes)")
else:
    print(f"  ⚠ {OPTIONAL_FILE} not found (optional - training will proceed without validation set)")


## Step P1-3.3: Setup Local Environment

Verify GPU availability, set up MLflow tracking (local file store), and check that key dependencies are installed. This step ensures the local environment is ready for training.


In [ ]:
import sys
import torch

DEFAULT_DEVICE = "cuda"

env_config = configs["env"]
device_type = env_config.get("compute", {}).get("device", DEFAULT_DEVICE)

if device_type == "cuda" and not torch.cuda.is_available():
    raise RuntimeError(
        "CUDA device requested but not available. "
        "Set device to 'cpu' in env config or ensure CUDA is properly installed."
    )


In [ ]:
import mlflow

MLFLOW_DIR = "mlruns"
mlflow_tracking_path = ROOT_DIR / MLFLOW_DIR
mlflow_tracking_path.mkdir(exist_ok=True)

# Convert Windows path to file:// URI format for MLflow
mlflow_tracking_uri = mlflow_tracking_path.as_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)


In [ ]:
try:
    import transformers
    import optuna
except ImportError as e:
    raise ImportError(f"Required package not installed: {e}")

REQUIRED_PACKAGES = {
    "torch": torch,
    "transformers": transformers,
    "mlflow": mlflow,
    "optuna": optuna,
}

for name, module in REQUIRED_PACKAGES.items():
    if not hasattr(module, "__version__"):
        raise ImportError(f"Required package '{name}' is not properly installed")


## Step P1-3.4: The Dry Run

Run a minimal HPO sweep to validate the training pipeline works correctly before launching the full HPO sweep. Uses the smoke HPO configuration with reduced trials.


In [ ]:
from pathlib import Path
from orchestration import STAGE_SMOKE
from orchestration.jobs.local_sweeps import run_local_hpo_sweep

TRAINING_SCRIPT_PATH = SRC_DIR / "train.py"
DRY_RUN_OUTPUT_DIR = ROOT_DIR / "outputs" / "dry_run"

if not TRAINING_SCRIPT_PATH.exists():
    raise FileNotFoundError(f"Training script not found: {TRAINING_SCRIPT_PATH}")


In [ ]:
hpo_config = configs["hpo"]
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]

dry_run_studies = {}

for backbone in backbone_values:
    mlflow_experiment_name = f"{experiment_config.name}-{STAGE_SMOKE}-{backbone}"
    backbone_output_dir = DRY_RUN_OUTPUT_DIR / backbone
    
    study = run_local_hpo_sweep(
        dataset_path=str(DATASET_LOCAL_PATH),
        config_dir=CONFIG_DIR,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_dir=backbone_output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
    )
    
    dry_run_studies[backbone] = study


In [ ]:
for backbone, study in dry_run_studies.items():
    if study.trials:
        best_trial = study.best_trial
        print(f"{backbone}: {len(study.trials)} trials completed")
        print(
            f"  Best {hpo_config['objective']['metric']}: {best_trial.value:.4f}")
        print(f"  Best params: {best_trial.params}")
    else:
        print(f"{backbone}: No trials completed")

## Step P1-3.5: The Sweep (HPO) - Local with Optuna

Run the full hyperparameter optimization sweep using Optuna to systematically search for the best model configuration. Uses the production HPO configuration with more trials than the dry run.

**Note on K-Fold Cross-Validation:**
- When k-fold CV is enabled (`k_fold.enabled: true`), each trial trains **k models** (one per fold) and returns the **average metric** across folds
- The number of **trials** is controlled by `sampling.max_trials` (e.g., 2 trials in smoke.yaml)
- With k=5 folds and 2 trials: **2 trials × 5 folds = 10 model trainings total**
- K-fold CV provides more robust hyperparameter evaluation but increases compute time (k× per trial)

In [ ]:
from pathlib import Path
from orchestration import STAGE_HPO
from shared.yaml_utils import load_yaml
from orchestration.jobs.local_sweeps import run_local_hpo_sweep

DEFAULT_K_FOLDS = 5
DEFAULT_RANDOM_SEED = 42
HPO_OUTPUT_DIR = ROOT_DIR / "outputs" / "hpo"

hpo_stage_config = experiment_config.stages.get(STAGE_HPO, {})
hpo_config_override = hpo_stage_config.get("hpo_config")
hpo_config_path = CONFIG_DIR / hpo_config_override if hpo_config_override else experiment_config.hpo_config

if not hpo_config_path.exists():
    raise FileNotFoundError(f"HPO config not found: {hpo_config_path}")


In [ ]:
hpo_config = load_yaml(hpo_config_path)
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]


In [ ]:
from training.cv_utils import create_kfold_splits, save_fold_splits
from training.data import load_dataset

k_fold_config = hpo_config.get("k_fold", {})
k_folds_enabled = k_fold_config.get("enabled", False)
fold_splits_file = None

if k_folds_enabled:
    n_splits = k_fold_config.get("n_splits", DEFAULT_K_FOLDS)
    random_seed = k_fold_config.get("random_seed", DEFAULT_RANDOM_SEED)
    shuffle = k_fold_config.get("shuffle", True)
    
    full_dataset = load_dataset(str(DATASET_LOCAL_PATH))
    train_data = full_dataset.get("train", [])
    
    fold_splits = create_kfold_splits(
        dataset=train_data,
        k=n_splits,
        random_seed=random_seed,
        shuffle=shuffle,
    )
    
    fold_splits_file = HPO_OUTPUT_DIR / "fold_splits.json"
    save_fold_splits(
        fold_splits,
        fold_splits_file,
        metadata={
            "k": n_splits,
            "random_seed": random_seed,
            "shuffle": shuffle,
            "dataset_path": str(DATASET_LOCAL_PATH),
        }
    )


In [ ]:
def build_mlflow_experiment_name(experiment_name: str, stage: str, backbone: str) -> str:
    return f"{experiment_name}-{stage}-{backbone}"

hpo_studies = {}
k_folds_param = k_fold_config.get("n_splits", DEFAULT_K_FOLDS) if k_folds_enabled else None

for backbone in backbone_values:
    mlflow_experiment_name = build_mlflow_experiment_name(
        experiment_config.name, STAGE_HPO, backbone
    )
    backbone_output_dir = HPO_OUTPUT_DIR / backbone
    
    study = run_local_hpo_sweep(
        dataset_path=str(DATASET_LOCAL_PATH),
        config_dir=CONFIG_DIR,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_dir=backbone_output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
        k_folds=k_folds_param,
        fold_splits_file=fold_splits_file,
    )
    
    hpo_studies[backbone] = study


In [ ]:
def extract_cv_statistics(best_trial):
    if not hasattr(best_trial, "user_attrs"):
        return None
    cv_mean = best_trial.user_attrs.get("cv_mean")
    cv_std = best_trial.user_attrs.get("cv_std")
    return (cv_mean, cv_std) if cv_mean is not None else None

objective_metric = hpo_config['objective']['metric']

for backbone, study in hpo_studies.items():
    if not study.trials:
        continue
    
    best_trial = study.best_trial
    cv_stats = extract_cv_statistics(best_trial)
    
    print(f"{backbone}: {len(study.trials)} trials completed")
    print(f"  Best {objective_metric}: {best_trial.value:.4f}")
    print(f"  Best params: {best_trial.params}")
    
    if cv_stats:
        cv_mean, cv_std = cv_stats
        print(f"  CV Statistics: Mean: {cv_mean:.4f} ± {cv_std:.4f}")


## Step P1-3.6: Best Configuration Selection (Automated)

Programmatically select the best configuration from all HPO sweep runs across all backbone models. The best configuration is determined by the objective metric specified in the HPO config.


In [ ]:
from pathlib import Path
from shared.json_cache import save_json
from orchestration.jobs.local_selection import select_best_configuration_across_studies

BEST_CONFIG_CACHE_FILE = ROOT_DIR / "notebooks" / "best_configuration_cache.json"


In [ ]:
dataset_version = data_config.get("version", "unknown")

best_configuration = select_best_configuration_across_studies(
    studies=hpo_studies,
    hpo_config=hpo_config,
    dataset_version=dataset_version,
)


In [ ]:
save_json(BEST_CONFIG_CACHE_FILE, best_configuration)

print(f"Best configuration selected:")
print(f"  Backbone: {best_configuration.get('backbone')}")
print(f"  Trial: {best_configuration.get('trial_name')}")
print(f"  Best {hpo_config['objective']['metric']}: {best_configuration.get('selection_criteria', {}).get('best_value'):.4f}")
print(f"  Hyperparameters: {best_configuration.get('hyperparameters')}")
print(f"\nSaved to: {BEST_CONFIG_CACHE_FILE}")


## Step P1-3.7: Final Training (Post-HPO, Single Run)

Train the final production model using the best configuration from HPO with stable, controlled conditions. This uses the full training epochs (no early stopping) and the best hyperparameters found during HPO.


In [ ]:
from pathlib import Path
import os
import sys
import subprocess
import mlflow
from shared.json_cache import load_json, save_json
from orchestration import STAGE_TRAINING
from orchestration.jobs.training import build_final_training_config

DEFAULT_RANDOM_SEED = 42
BEST_CONFIG_CACHE_FILE = ROOT_DIR / "notebooks" / "best_configuration_cache.json"
FINAL_TRAINING_OUTPUT_DIR = ROOT_DIR / "outputs" / "final_training"


In [ ]:
best_configuration = load_json(BEST_CONFIG_CACHE_FILE, default=None)

if best_configuration is None:
    raise FileNotFoundError(
        f"Best configuration cache not found: {BEST_CONFIG_CACHE_FILE}\n"
        f"Please run Step P1-3.6: Best Configuration Selection first."
    )

final_training_config = build_final_training_config(
    best_config=best_configuration,
    train_config=configs["train"],
    random_seed=DEFAULT_RANDOM_SEED,
)


In [ ]:
mlflow_experiment_name = f"{experiment_config.name}-{STAGE_TRAINING}-{final_training_config['backbone']}"
final_output_dir = FINAL_TRAINING_OUTPUT_DIR / final_training_config['backbone']
final_output_dir.mkdir(parents=True, exist_ok=True)

mlflow.set_experiment(mlflow_experiment_name)


In [ ]:
training_script_path = SRC_DIR / "train.py"
training_args = [
    sys.executable,
    str(training_script_path),
    "--data-asset",
    str(DATASET_LOCAL_PATH),
    "--config-dir",
    str(CONFIG_DIR),
    "--backbone",
    final_training_config["backbone"],
    "--learning-rate",
    str(final_training_config["learning_rate"]),
    "--batch-size",
    str(final_training_config["batch_size"]),
    "--dropout",
    str(final_training_config["dropout"]),
    "--weight-decay",
    str(final_training_config["weight_decay"]),
    "--epochs",
    str(final_training_config["epochs"]),
    "--random-seed",
    str(final_training_config["random_seed"]),
    "--early-stopping-enabled",
    str(final_training_config["early_stopping_enabled"]).lower(),
    "--use-combined-data",
    str(final_training_config["use_combined_data"]).lower(),
]


In [ ]:
training_env = os.environ.copy()
training_env["AZURE_ML_OUTPUT_checkpoint"] = str(final_output_dir)

mlflow_tracking_uri = mlflow.get_tracking_uri()
if mlflow_tracking_uri:
    training_env["MLFLOW_TRACKING_URI"] = mlflow_tracking_uri
training_env["MLFLOW_EXPERIMENT_NAME"] = mlflow_experiment_name


In [ ]:
result = subprocess.run(
    training_args,
    cwd=ROOT_DIR,
    env=training_env,
    capture_output=False,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Final training failed with return code {result.returncode}")


In [ ]:
import json

METRICS_FILENAME = "metrics.json"
FINAL_TRAINING_CACHE_FILE = ROOT_DIR / "notebooks" / "final_training_cache.json"

metrics_file = final_output_dir / METRICS_FILENAME
if metrics_file.exists():
    with open(metrics_file, "r") as f:
        metrics = json.load(f)
    print(f"✓ Training completed. Checkpoint: {final_output_dir}")
else:
    raise FileNotFoundError(f"Training metrics not found: {metrics_file}")

save_json(FINAL_TRAINING_CACHE_FILE, {
    "output_dir": str(final_output_dir),
    "backbone": final_training_config["backbone"],
    "config": final_training_config,
})


## Step P1-4: Model Conversion & Optimization

Convert the final training checkpoint to an optimized ONNX model (int8 quantized) for production inference.

**Platform Adapter Note**: The conversion script (`src/convert_to_onnx.py`) uses the platform adapter to automatically handle output paths and logging appropriately for local execution.


In [ ]:
from pathlib import Path
import os
import sys
import subprocess
import mlflow
from shared.json_cache import load_json

CONVERSION_SCRIPT_PATH = SRC_DIR / "convert_to_onnx.py"
FINAL_TRAINING_CACHE_FILE = ROOT_DIR / "notebooks" / "final_training_cache.json"
CONVERSION_OUTPUT_DIR = ROOT_DIR / "outputs" / "conversion"


In [ ]:
training_cache = load_json(FINAL_TRAINING_CACHE_FILE, default=None)

if training_cache is None:
    raise FileNotFoundError(
        f"Final training cache not found: {FINAL_TRAINING_CACHE_FILE}\n"
        f"Please run Step P1-3.7: Final Training first."
    )

checkpoint_dir = Path(training_cache["output_dir"]) / "checkpoint"
if not checkpoint_dir.exists():
    raise FileNotFoundError(f"Checkpoint directory not found: {checkpoint_dir}")

backbone = training_cache["backbone"]
conversion_output_dir = CONVERSION_OUTPUT_DIR / backbone
conversion_output_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
conversion_args = [
    sys.executable,
    str(CONVERSION_SCRIPT_PATH),
    "--checkpoint-path",
    str(checkpoint_dir),
    "--config-dir",
    str(CONFIG_DIR),
    "--backbone",
    backbone,
    "--output-dir",
    str(conversion_output_dir),
    "--quantize-int8",
    "--run-smoke-test",
]


In [ ]:
conversion_env = os.environ.copy()
conversion_env["AZURE_ML_OUTPUT_onnx_model"] = str(conversion_output_dir)

mlflow_tracking_uri = mlflow.get_tracking_uri()
if mlflow_tracking_uri:
    conversion_env["MLFLOW_TRACKING_URI"] = mlflow_tracking_uri


In [ ]:
result = subprocess.run(
    conversion_args,
    cwd=ROOT_DIR,
    env=conversion_env,
    capture_output=False,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Model conversion failed with return code {result.returncode}")


In [ ]:
from shared.json_cache import save_json

ONNX_MODEL_FILENAME = "model_int8.onnx"
FALLBACK_ONNX_MODEL_FILENAME = "model.onnx"
CONVERSION_CACHE_FILE = ROOT_DIR / "notebooks" / "conversion_cache.json"

onnx_model_path = conversion_output_dir / ONNX_MODEL_FILENAME
if not onnx_model_path.exists():
    onnx_model_path = conversion_output_dir / FALLBACK_ONNX_MODEL_FILENAME

if not onnx_model_path.exists():
    raise FileNotFoundError(f"ONNX model not found in {conversion_output_dir}")

print(f"✓ Conversion completed. ONNX model: {onnx_model_path}")

save_json(CONVERSION_CACHE_FILE, {
    "onnx_model_path": str(onnx_model_path),
    "backbone": backbone,
    "checkpoint_dir": str(checkpoint_dir),
})
